In [1]:
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle, datetime

SCOPES = ['https://www.googleapis.com/auth/calendar']
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

In [22]:
from pymongo import MongoClient
from utils_mongo import connect_to_mongo, save_new, update_event, delete_event, get_event 
from utils import CalendarTool

import re
from datetime import datetime
import pytz

In [4]:
result2 = """{
"event_summary": "Team meeting",
"event_location": "N/A",
"start_time": "2024-03-02T20:00:00+05:30",
"end_time": "2024-03-02T22:00:00+05:30",
"email_reminder_minutes": 1440,
"popup_reminder_minutes": 180
}"""

result_dict = eval(result2)
print(result_dict)

{'event_summary': 'Team meeting', 'event_location': 'N/A', 'start_time': '2024-03-02T20:00:00+05:30', 'end_time': '2024-03-02T22:00:00+05:30', 'email_reminder_minutes': 1440, 'popup_reminder_minutes': 180}


In [6]:
created_event = CalendarTool.create_event(service, result_dict.get('event_summary'), result_dict.get('event_location'), result_dict.get('start_time'), result_dict.get('end_time'), int(result_dict.get('email_reminder_minutes')), int(result_dict.get('popup_reminder_minutes')))
eventId = re.sub(r'@.*', '', created_event)
print(eventId)

event = CalendarTool.list_events(service, event_id=eventId)

Event created: https://www.google.com/calendar/event?eid=ZW11bHNwYjA1Y2Z0ZTVjcTVnODJ2YTlrbTAgZ3VudWJhbnNhbDEyOUBt
emulspb05cfte5cq5g82va9km0


In [11]:
collection = connect_to_mongo("agent_calendar", "exec_ai")

In [23]:
save_new(event= event[0], collection=collection)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\gunub\.conda\envs\langchain\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\gunub\AppData\Local\Temp\ipykernel_30792\3611410648.py", line 1, in <module>
    save_new(event= event[0], collection=collection)
  File "d:\C_Drive\Desktop\CS\crewAI\utils_mongo.py", line 11, in save_new
  File "c:\Users\gunub\.conda\envs\langchain\Lib\site-packages\pymongo\collection.py", line 671, in insert_one
    self._insert_one(
  File "c:\Users\gunub\.conda\envs\langchain\Lib\site-packages\pymongo\collection.py", line 611, in _insert_one
    self.__database.client._retryable_write(acknowledged, _insert_command, session)
  File "c:\Users\gunub\.conda\envs\langchain\Lib\site-packages\pymongo\mongo_client.py", line 1568, in _retryable_write
    return self._retry_with_session(retryable, func, s, None)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [16]:
from datetime import datetime
import pytz